<a href="https://colab.research.google.com/github/SameehanSahasrabudhe/CA2_ML2_DL/blob/main/Gender_and_age_detection_TrasferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gender and age detector using functional API using VGG and finetuning the FF network

In [26]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [27]:
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
utkface-new.zip: Skipping, found more recently modified local copy (use --force to force download)


In [28]:
import zipfile
zip_ref = zipfile.ZipFile('/content/utkface-new.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()


In [29]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
# from keras.preprocessing.image import ImageDataGenerator

In [30]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

ages = []
genders = []
images = []

for img_name in os.listdir(folder_path):
  img_path = os.path.join(folder_path, img_name)
  ages.append(int(img_name.split('_')[0]))
  genders.append(int(img_name.split('_')[1]))
  images.append(img_name)

df = pd.DataFrame({'ages':ages, 'genders':genders, 'images':images})
df.head()

,ages,genders,images
0,21,1,21_1_1_20170112191219486.jpg.chip.jpg
1,37,0,37_0_0_20170104201816626.jpg.chip.jpg
2,30,0,30_0_0_20170104201841050.jpg.chip.jpg
3,32,1,32_1_0_20170117134826702.jpg.chip.jpg
4,29,1,29_1_3_20170116162100478.jpg.chip.jpg


In [31]:
train_ds = df.sample(frac=1, random_state=42).iloc[:20000]
validation_ds = df.sample(frac=1, random_state=42).iloc[18000:20000]
test_ds = df.sample(frac=1, random_state=42).iloc[20000:]

In [32]:
import tensorflow as tf
def load_and_process_image(image_path, age, gender):
  img = tf.io.read_file(image_path)
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.resize(img, [224,224])
  img = img/255
  return img, (age, gender)

# Training dataset
train_image_paths = folder_path + '/' + train_ds['images']
train_ages = train_ds['ages']
train_genders = train_ds['genders']

training_dataset = tf.data.Dataset.from_tensor_slices((train_image_paths, train_ages, train_genders))
training_dataset = training_dataset.map(load_and_process_image).batch(32)

# Validation dataset
val_images_path = folder_path + '/' + validation_ds['images']
val_ages = validation_ds['ages']
val_genders = validation_ds['genders']

val_dataset = tf.data.Dataset.from_tensor_slices((val_images_path, val_ages, val_genders))
val_dataset = val_dataset.map(load_and_process_image).batch(32)

# Test dataset
test_image_paths = folder_path + '/' + test_ds['images']
test_ages = test_ds['ages']
test_genders = test_ds['genders']

test_dataset = tf.data.Dataset.from_tensor_slices((test_image_paths, test_ages, test_genders))
test_dataset = test_dataset.map(load_and_process_image).batch(32)

In [33]:
from keras.applications.vgg16 import VGG16
from keras.layers import *
from keras.models import Model

In [34]:
vggnet = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

In [35]:
vggnet.trainable = False

output = vggnet.layers[-1].output

flatten = Flatten()(output)

hidden1 = Dense(512, activation = 'relu')(flatten)
norm1 = BatchNormalization()(hidden1)
drop1 = Dropout(0.2)(norm1)
hidden2 = Dense(512, activation = 'relu')(flatten)
norm2 = BatchNormalization()(hidden2)
drop2 = Dropout(0.2)(norm2)

hidden3 = Dense(512, activation='relu')(drop1)
hidden4 = Dense(512, activation='relu')(drop2)

output1 = Dense(1, activation='linear', name = 'age')(hidden3)
output2 = Dense(1, activation='sigmoid', name = 'gender')(hidden4)

model = Model(inputs=vggnet.input, outputs=[output1, output2])

In [36]:

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 224, 224,  │      1,792 │ input_layer_1[0]… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 224, 224,  │     36,928 │ block1_conv1[0][… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_pool         │ (None, 112, 112,  │          0 │ block1_conv2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv1        │ (None, 112, 112,  │     73,856 │ block1_pool[0][0] │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv2        │ (None, 112, 112,  │    147,584 │ block2_conv1[0][… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 56, 56,    │          0 │ block2_conv2[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv1        │ (None, 56, 56,    │    295,168 │ block2_pool[0][0] │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv2        │ (None, 56, 56,    │    590,080 │ block3_conv1[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv3        │ (None, 56, 56,    │    590,080 │ block3_conv2[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_pool         │ (None, 28, 28,    │          0 │ block3_conv3[0][… │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv1        │ (None, 28, 28,    │  1,180,160 │ block3_pool[0][0] │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv2        │ (None, 28, 28,    │  2,359,808 │ block4_conv1[0][… │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv3        │ (None, 28, 28,    │  2,359,808 │ block4_conv2[0][… │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_pool         │ (None, 14, 14,    │          0 │ block4_conv3[0][… │
│ (MaxPooling2D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5_conv1        │ (None, 14, 14,    │  2,359,808 │ block4_pool[0][0] │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5_conv2        │ (None, 14, 14,    │  2,359,808 │ block5_conv1[0][

 Total params: 40,936,258 (156.16 MB)

 Trainable params: 26,219,522 (100.02 MB)

 Non-trainable params: 14,716,736 (56.14 MB)

In [37]:
# !apt-get install graphviz
# from keras.utils import plot_model
# plot_model(model, show_shapes=True)

In [38]:
model.compile(optimizer='adam',
              loss = {'age':'mae', 'gender':'binary_crossentropy'},
              metrics = {'age':'mae', 'gender':'accuracy'})

In [ ]:
history = model.fit(training_dataset, batch_size=32, epochs=5, validation_data=test_dataset)

Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 133s 205ms/step - age_loss: 12.6184 - age_mae: 12.6184 - gender_accuracy: 0.8125 - gender_loss: 0.4192 - loss: 13.0376 - val_age_loss: 8.1430 - val_age_mae: 8.1458 - val_gender_accuracy: 0.8630 - val_gender_loss: 0.2939 - val_loss: 8.4398
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 127s 203ms/step - age_loss: 7.3936 - age_mae: 7.3936 - gender_accuracy: 0.8903 - gender_loss: 0.2524 - loss: 7.6460 - val_age_loss: 7.9895 - val_age_mae: 7.9918 - val_gender_accuracy: 0.8773 - val_gender_loss: 0.3218 - val_loss: 8.3138
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 142s 204ms/step - age_loss: 6.7010 - age_mae: 6.7010 - gender_accuracy: 0.9199 - gender_loss: 0.1950 - loss: 6.8960 - val_age_loss: 6.9740 - val_age_mae: 6.9757 - val_gender_accuracy: 0.8730 - val_gender_loss: 0.3315 - val_loss: 7.3074
Epoch 4/5
145/625 ━━━━━━━━━━━━━━━━━━━━ 1:22 172ms/step - age_loss: 6.3825 - age_mae: 6.3825 - gender_accuracy: 0.9325 - gender_loss: 0.1679 - loss: 6.5504

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['age_loss'], label='Training age Loss')
plt.plot(history.history['val_age_loss'], label='Validation age Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


plt.plot(history.history['gender_loss'], label = 'Training gender Loss')
plt.plot(history.history['val_gender_loss'], label = 'Validation gender Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['age_mae'], label='Training age MAE')
plt.plot(history.history['val_age_mae'], label='Validation age MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()

plt.plot(history.history['gender_accuracy'], label = 'Training gender Accuracy')
plt.plot(history.history['val_gender_accuracy'], label = 'Validation gender Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()